#Todo: Add more comments


# Lecture 12. Intro to TensorFlow
To be run on google colab.

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

from keras.models import Sequential
from keras.layers import Dense,Dropout
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

import os
import warnings

from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch  # pip install keras-tuner
warnings.filterwarnings('ignore')

import shap

2024-06-18 16:05:33.236620: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-18 16:05:33.238569: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-18 16:05:33.262308: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-18 16:05:33.854828: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/tmp/ipykernel_57049/4032085288.py:14: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [2]:
# pip install tensorflow keras keras-tuner shap

In [3]:
train_data = pd.read_csv('har_train.csv')
test_data = pd.read_csv('har_test.csv')

print(f'Shape of train data is: {train_data.shape}\nShape of test data is: {test_data.shape}')

Shape of train data is: (7352, 563)
Shape of test data is: (2947, 563)


In [4]:


x_train, y_train = train_data.iloc[:, :-2], train_data.iloc[:, -1:]
x_test, y_test = test_data.iloc[:, :-2], test_data.iloc[:, -1:]
x_train.shape, y_train.shape



((7352, 561), (7352, 1))

In [5]:
x_test, y_test = test_data.iloc[:, :-2], test_data.iloc[:, -1:]
x_test.shape, y_test.shape

((2947, 561), (2947, 1))

In [6]:
y_train

,Activity
0,STANDING
1,STANDING
2,STANDING
3,STANDING
4,STANDING
...,...
7347,WALKING_UPSTAIRS
7348,WALKING_UPSTAIRS
7349,WALKING_UPSTAIRS
7350,WALKING_UPSTAIRS


In [7]:
le = LabelEncoder()

In [8]:
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

In [9]:
y_train

array([2, 2, 2, ..., 5, 5, 5])

In [10]:
scaling_data = MinMaxScaler()

x_train = scaling_data.fit_transform(x_train)
x_test = scaling_data.transform(x_test)

In [11]:
x_train

array([[0.64429225, 0.48985291, 0.43354743, ..., 0.07937662, 0.79825103,
        0.47068654],
       [0.63920942, 0.49179472, 0.4382399 , ..., 0.0776062 , 0.79848665,
        0.47284164],
       [0.63982653, 0.49026642, 0.44326915, ..., 0.07553327, 0.79872236,
        0.47544109],
       ...,
       [0.63669369, 0.49149469, 0.47748909, ..., 0.11043369, 0.84506893,
        0.52040559],
       [0.64482708, 0.49057848, 0.42085971, ..., 0.10740929, 0.84323381,
        0.51266974],
       [0.67575173, 0.49378844, 0.39806642, ..., 0.10836653, 0.84348837,
        0.51834742]])

In [12]:
type(x_train)

numpy.ndarray

### Model

In [13]:
model = Sequential()

In [14]:
# Sequential??

In [15]:
x_train.shape[1]

561

In [16]:
model.add(
    Dense(
    units=64,
    kernel_initializer='normal',
    activation='sigmoid',
    input_dim=x_train.shape[1])
)

2024-06-18 16:05:35.039394: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-18 16:05:35.039754: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [17]:
model

<Sequential name=sequential, built=True>

In [18]:
model.add(Dropout(0.2))

In [19]:
model.add(keras.layers.Dense(64, activation='relu')) 

In [20]:
model.add(keras.layers.Dense(64, activation='relu')) 

In [21]:
# model.add(keras.layers.Dense(64,
#                              activation='relu',
#                              kernel_regularizer=regularizers.l1(0.01)))

In [22]:
# model.add(Dense(64, activation='relu')) 

In [23]:
model.add(Dense(units=6,
                kernel_initializer='uniform',
                activation='softmax'))

In [24]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [25]:
history = model.fit(
    x_train, y_train, 
    batch_size = 64, 
    epochs= 10,
    validation_data = (x_test,y_test))

Epoch 1/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3310 - loss: 1.4746 - val_accuracy: 0.5816 - val_loss: 0.9034
Epoch 2/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 827us/step - accuracy: 0.6643 - loss: 0.7506 - val_accuracy: 0.8056 - val_loss: 0.4729
Epoch 3/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 845us/step - accuracy: 0.8327 - loss: 0.3913 - val_accuracy: 0.8534 - val_loss: 0.3347
Epoch 4/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 815us/step - accuracy: 0.8649 - loss: 0.3213 - val_accuracy: 0.9013 - val_loss: 0.2631
Epoch 5/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 813us/step - accuracy: 0.8864 - loss: 0.2655 - val_accuracy: 0.8856 - val_loss: 0.2761
Epoch 6/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 818us/step - accuracy: 0.9088 - loss: 0.2199 - val_accuracy: 0.9250 - val_loss: 0.2015
Epoch 7/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step - accuracy: 0.9171 - loss: 0.2031 - val_accuracy: 0.9274 - val_loss: 0.2032
Epoch 8/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 809us/step - accuracy: 0.9343 - loss: 0.1688 - va

In [26]:
 help(model.add)

Help on method add in module keras.src.models.sequential:

add(layer, rebuild=True) method of keras.src.models.sequential.Sequential instance
    Adds a layer instance on top of the layer stack.
    
    Args:
        layer: layer instance.



In [27]:
model.fit(
    x_train, y_train, 
    batch_size = 64, 
    epochs= 10,
    validation_data = (x_test,y_test))

Epoch 1/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 991us/step - accuracy: 0.9441 - loss: 0.1355 - val_accuracy: 0.9338 - val_loss: 0.1705
Epoch 2/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 801us/step - accuracy: 0.9461 - loss: 0.1358 - val_accuracy: 0.9260 - val_loss: 0.2017
Epoch 3/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 814us/step - accuracy: 0.9526 - loss: 0.1173 - val_accuracy: 0.9220 - val_loss: 0.1923
Epoch 4/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 825us/step - accuracy: 0.9529 - loss: 0.1183 - val_accuracy: 0.9423 - val_loss: 0.1461
Epoch 5/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 807us/step - accuracy: 0.9594 - loss: 0.1066 - val_accuracy: 0.9223 - val_loss: 0.1942
Epoch 6/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 794us/step - accuracy: 0.9604 - loss: 0.1051 - val_accuracy: 0.9372 - val_loss: 0.1536
Epoch 7/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 807us/step - accuracy: 0.9612 - loss: 0.0965 - val_accuracy: 0.9379 - val_loss: 0.1798
Epoch 8/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 793us/step - accuracy: 0.9585 - loss: 0.0988 - 

In [28]:
# # we use the first 100 training examples as our background dataset to integrate over
# explainer = shap.DeepExplainer(model, x_train)

# # explain the first 10 predictions
# # explaining each prediction requires 2 * background dataset size runs
# shap_values = explainer.shap_values(x_test)



In [29]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 25)):
        model.add(layers.Dense(units = hp.Int('units' + str(i), min_value=32, max_value=512, step=32),
                               kernel_initializer= hp.Choice('initializer', ['uniform', 'normal']),
                               activation= hp.Choice('activation', ['relu', 'sigmoid', 'tanh'])))
    model.add(layers.Dense(6, kernel_initializer= hp.Choice('initializer', ['uniform', 'normal']), activation='softmax'))
    model.add(
            Dropout(0.2))
    model.compile(
        optimizer = 'adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model


tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials= 5,
    executions_per_trial=3,
    directory='project', project_name = 'Human_activity_recognition')

tuner.search_space_summary()

Reloading Tuner from project/Human_activity_recognition/tuner0.json
Search space summary
Default search space size: 27
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 25, 'step': 1, 'sampling': 'linear'}
units0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
initializer (Choice)
{'default': 'uniform', 'conditions': [], 'values': ['uniform', 'normal'], 'ordered': False}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid', 'tanh'], 'ordered': False}
units1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units4 (Int)
{'default': None, 'conditions': [], 'min_value': 

In [30]:
tuner.search(x_train, y_train,
             epochs= 10,
             validation_data=(x_test, y_test))

In [31]:


model.summary()
import tensorflow as tf
from tensorflow import keras

Callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)
mo_fitt = model.fit(x_train,y_train, epochs=200, validation_data=(x_test,y_test), callbacks=Callback)



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │        35,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,036 (523.58 KB)

 Trainable params: 44,678 (174.52 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 89,358 (349.06 KB)

Epoch 1/200
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 815us/step - accuracy: 0.9618 - loss: 0.1077 - val_accuracy: 0.9199 - val_loss: 0.2219
Epoch 2/200
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 705us/step - accuracy: 0.9580 - loss: 0.1028 - val_accuracy: 0.9491 - val_loss: 0.1275
Epoch 3/200
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 717us/step - accuracy: 0.9675 - loss: 0.0788 - val_accuracy: 0.9494 - val_loss: 0.1372
Epoch 4/200
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 711us/step - accuracy: 0.9702 - loss: 0.0771 - val_accuracy: 0.9474 - val_loss: 0.1359
Epoch 5/200
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 712us/step - accuracy: 0.9670 - loss: 0.0807 - val_accuracy: 0.9389 - val_loss: 0.1647
Epoch 6/200
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 700us/step - accuracy: 0.9643 - loss: 0.0873 - val_accuracy: 0.9382 - val_loss: 0.1714
Epoch 7/200
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 726us/step - accuracy: 0.9672 - loss: 0.0809 - val_accuracy: 0.9403 - val_loss: 0.1784
Epoch 8/200
230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 735us/step - accuracy: 0.9727 - loss: 0

https://www.kaggle.com/code/fahadmehfoooz/human-activity-recognition-with-neural-networks